# Example 5: Stickleback morphometrics - body and armor-plate shape

Variation in continuous phenotypic traits like shape or area of certain structures are difficult to quantify with landmarks, because they are too complex or have no underlying assumption of homology. In this example, the number and area of armor plating was measured as a continuous trait in a two-step process: first, a mask was set around the posterior region that contains the plates, second, the red channel (highest signal-to-noise-ratio) of the image was thresholded. 

## High throughput workflow

First we need to create a project, as described in [Tutorial 3](tutorial_3_managing_projects_1.ipynb) and [Tutorial 4](tutorial_4_managing_projects_2.ipynb). This will create a folder structure and allows for easy creation of the `pype`-configuration files needed for high throughput. 

In [2]:
import phenopype as pp

In [1]:
## relative from now (phenopype-master/tutorials)
project_root = r"../_temp/output/ex5_project"
image_dir = "images"

In [3]:
ex5 = pp.project(root_dir=project_root)

--------------------------------------------
Phenopype will create a new project at
E:\git_repos\phenopype\_temp\output\ex5_project

Proceed? (y/n)
y
Warning - project root_dir already exists - overwrite? (y/n)y

"E:\git_repos\phenopype\_temp\output\ex5_project" created (overwritten)

project attributes written to E:\git_repos\phenopype\_temp\output\ex5_project\attributes.yaml
--------------------------------------------


First we add the image files in the directory, but only "stickle1", "stickle2", and "stickle3". Then we add the appropriate configuration file. As for the other examples I have created a preset ("ex5") with appropriate settings, which is passed to the pype using `config_preset="ex5"`.

In [4]:
ex5.add_files(image_dir=image_dir, include="stickle", exclude=["side","top"])

--------------------------------------------
phenopype will search for files at

E:\git_repos\phenopype\tutorials\images

using the following settings:

filetypes: ['jpg', 'JPG', 'jpeg', 'JPEG', 'tif', 'png'], include: stickle, exclude: ['side', 'top'], raw_mode: copy, search_mode: dir, unique_mode: path

Found image stickle1.JPG - phenopype-project folder 0__stickle1 created
dirpath defaulted to file directory - E:\git_repos\phenopype\tutorials\images
Directory to save files set at - E:\git_repos\phenopype\tutorials\images
no meta-data found
Found image stickle2.JPG - phenopype-project folder 0__stickle2 created
dirpath defaulted to file directory - E:\git_repos\phenopype\tutorials\images
Directory to save files set at - E:\git_repos\phenopype\tutorials\images
no meta-data found
Found image stickle3.JPG - phenopype-project folder 0__stickle3 created
dirpath defaulted to file directory - E:\git_repos\phenopype\tutorials\images
Directory to save files set at - E:\git_repos\phenopype\tut

In [5]:
ex5.add_config(name = "v1", interactive=True)


No preset provided - defaulting to preset preset1
dirpath defaulted to file directory - E:\git_repos\phenopype\_temp\output\ex5_project
Directory to save files set at - E:\git_repos\phenopype\_temp\output\ex5_project
E:\git_repos\phenopype\_temp\output\ex5_project\pype_config_template-v1.yaml


------------+++ new pype iteration 2020:05:17 14:09:37 +++--------------


SEGMENTATION
threshold
find_contours
VISUALIZATION
select_canvas
- invalid selection - defaulting to raw image
draw_contours


------------+++ new pype iteration 2020:05:17 14:09:44 +++--------------


PREPROCESSING
create_mask
- create mask
SEGMENTATION
blur
threshold
- include mask "mask1" pixels
morphology
watershed
find_contours
VISUALIZATION
select_canvas
- binary image
draw_contours


------------+++ new pype iteration 2020:05:17 14:10:04 +++--------------


PREPROCESSING
create_mask
- mask with label mask1 already created (overwrite=False)
SEGMENTATION
blur
threshold
- include mask "mask1" pixels
morphology
watersh

SystemExit: 

TERMINATE (by user)